In [1]:
import torch
import warnings
warnings.filterwarnings("ignore")
def check_memory(gpu_index: int = 0):
    if torch.cuda.is_available():
        free_memory = torch.cuda.get_device_properties(gpu_index).total_memory - torch.cuda.memory_allocated(gpu_index)
        total_memory = torch.cuda.get_device_properties(gpu_index).total_memory
        print(f"Free GPU Memory: {free_memory / 1024**3:.2f} GB")
        print(f"Total GPU Memory: {total_memory / 1024**3:.2f} GB")
    else:
        print("CUDA is not available.")

In [2]:
def refresh_repo():
    %cd /kaggle/working
    %rm -rf hotflip
    !git clone https://github.com/jefri021/hotflip.git
    %cd /kaggle/working/hotflip/
    !git pull origin main

refresh_repo()

/kaggle/working
Cloning into 'hotflip'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 83 (delta 28), reused 75 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 3.47 MiB | 3.32 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/kaggle/working/hotflip
From https://github.com/jefri021/hotflip
 * branch            main       -> FETCH_HEAD
Already up to date.


In [3]:
import gc
import torch
from typing import List, Union

def clear_memory(keep_vars: Union[List[str], None] = None, verbose: bool = True):
    """
    Clears memory while preserving specified variables.
    Still clears GPU memory for all CUDA objects, including kept variables.
    
    Args:
        keep_vars: List of variable names to preserve in memory (will still be cleared from GPU)
        verbose: Whether to print memory clearing information
    """
    if verbose:
        print("Starting memory clearing process...")
    
    # Convert keep_vars to set for faster lookups
    keep_set = set(keep_vars) if keep_vars else set()
    
    # First pass: Move kept CUDA variables to CPU
    if torch.cuda.is_available():
        for name, var in list(globals().items()):
            if name in keep_set and isinstance(var, torch.Tensor) and var.is_cuda:
                if verbose:
                    print(f"Moving kept tensor '{name}' to CPU")
                globals()[name] = var.cpu()
    
    # Clear Python garbage collector
    gc.collect()
    if verbose:
        print("Ran Python garbage collection")
    
    # Clear CUDA memory if available
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        if verbose:
            print("Cleared CUDA cache")
            print(f"Current CUDA memory allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
            print(f"Current CUDA memory cached: {torch.cuda.memory_reserved()/1024**2:.2f} MB")
    
    # Try to clear TensorFlow/Keras if available
    try:
        import tensorflow as tf
        tf.keras.backend.clear_session()
        if verbose:
            print("Cleared TensorFlow/Keras session")
    except ImportError:
        pass
    
    # Delete objects not in keep_vars
    for name, var in list(globals().items()):
        if not name.startswith('__') and name not in keep_set:
            if isinstance(var, (torch.Tensor, torch.nn.Module)):
                del globals()[name]
                if verbose:
                    print(f"Deleted torch object: {name}")
            elif isinstance(var, list) and var and isinstance(var[0], torch.Tensor):
                del globals()[name]
                if verbose:
                    print(f"Deleted list of torch tensors: {name}")
    
    # Final garbage collection
    gc.collect()
    
    if verbose:
        print("Memory clearing complete")

In [4]:
import os
import gdown

def download_file_from_google_drive(file_id, output_dir, output_filename, quiet=False):
    """
    Downloads a file from Google Drive given its file ID and saves it to the specified directory.
    
    Args:
        file_id (str): The Google Drive file ID (found in the file URL)
        output_dir (str): Directory where the file should be saved
        output_filename (str): Name of the output file
        quiet (bool): Whether to suppress gdown output (default: False)
    
    Returns:
        str: Path to the downloaded file if successful, None otherwise
    """
    # Create directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Full output path
    output_file = os.path.join(output_dir, output_filename)
    
    print("Downloading the file...")
    try:
        gdown.download(id=file_id, output=output_file, quiet=quiet, fuzzy=True)
    except Exception as e:
        print(f"Download failed: {str(e)}")
        return None
    
    # Verify download
    if os.path.exists(output_file):
        file_size = os.path.getsize(output_file) / (1024 * 1024)  # in MB
        print(f"Download successful! File saved to: {output_file}")
        print(f"File size: {file_size:.2f} MB")
        return output_file
    else:
        print("Download failed - file not found")
        return None

In [5]:
import os
import tarfile
from typing import List, Union

def extract_and_delete_tar_gz(file_path: str, delete_compressed: bool = True) -> bool:
    """
    Extracts a .tar.gz file and optionally deletes the compressed file.
    
    Args:
        file_path (str): Path to the .tar.gz file
        delete_compressed (bool): Whether to delete the compressed file after extraction (default: True)
    
    Returns:
        bool: True if extraction was successful, False otherwise
    """
    try:
        print(f"Extracting: {file_path}")
        with tarfile.open(file_path, 'r:gz') as tar:
            tar.extractall(path=os.path.dirname(file_path))
        
        if delete_compressed:
            os.remove(file_path)
            print(f"Deleted compressed file: {file_path}")
        return True
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return False

def process_directory(directory: str, recursive: bool = True, max_depth: Union[int, None] = None) -> int:
    """
    Processes a directory to find and extract .tar.gz files.
    
    Args:
        directory (str): Directory path to process
        recursive (bool): Whether to process subdirectories (default: True)
        max_depth (int|None): Maximum recursion depth (None for unlimited)
    
    Returns:
        int: Number of .tar.gz files processed
    """
    processed_count = 0
    current_depth = 0
    
    while True:
        found_tar_gz = False
        for root, dirs, files in os.walk(directory):
            # Calculate current depth
            rel_path = os.path.relpath(root, directory)
            current_depth = rel_path.count(os.sep) + 1 if rel_path != '.' else 0
            
            # Skip if beyond max depth
            if max_depth is not None and current_depth > max_depth:
                continue
                
            for file in files:
                if file.endswith('.tar.gz'):
                    file_path = os.path.join(root, file)
                    if extract_and_delete_tar_gz(file_path):
                        processed_count += 1
                        found_tar_gz = True
        
        # If not recursive or no more .tar.gz files found, exit
        if not recursive or not found_tar_gz:
            break
    
    return processed_count

def process_paths(paths: List[str], recursive: bool = True, max_depth: Union[int, None] = None) -> int:
    """
    Processes a list of paths (files or directories) to extract .tar.gz files.
    
    Args:
        paths (List[str]): List of file/directory paths to process
        recursive (bool): Whether to process directories recursively (default: True)
        max_depth (int|None): Maximum recursion depth for directories (None for unlimited)
    
    Returns:
        int: Total number of .tar.gz files processed
    """
    total_processed = 0
    
    for path in paths:
        if not os.path.exists(path):
            print(f"Warning: Path does not exist - {path}")
            continue
            
        if path.endswith('.tar.gz'):
            if extract_and_delete_tar_gz(path):
                total_processed += 1
        elif os.path.isdir(path):
            print(f"Processing directory: {path}")
            total_processed += process_directory(
                directory=path,
                recursive=recursive,
                max_depth=max_depth
            )
    
    print(f"Total .tar.gz files processed: {total_processed}")
    return total_processed

In [6]:
from os.path import join

import torch
import json
import os
import logging
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer


def load_model(model_filepath: str, torch_dtype:torch.dtype=torch.float16):
    """Load a model given a specific model_path.

    Args:
        model_filepath: str - Path to where the model is stored

    Returns:
        model, dict, str - Torch model + dictionary representation of the model + model class name
    """

    conf_filepath = os.path.join(model_filepath, 'reduced-config.json')
    logging.info("Loading config file from: {}".format(conf_filepath))
    with open(conf_filepath, 'r') as fh:
        round_config = json.load(fh)

    logging.info("Loading model from filepath: {}".format(model_filepath))
    # https://huggingface.co/docs/transformers/installation#offline-mode
    if round_config['use_lora']:
        base_model_filepath = os.path.join(model_filepath, 'base-model')
        logging.info("loading the base model (before LORA) from {}".format(base_model_filepath))
        model = AutoModelForCausalLM.from_pretrained(base_model_filepath, device_map = "auto", trust_remote_code=True, torch_dtype=torch_dtype, local_files_only=True)
        # model = AutoModelForCausalLM.from_pretrained(round_config['model_architecture'], trust_remote_code=True, attn_implementation="flash_attention_2", torch_dtype=torch_dtype)

        fine_tuned_model_filepath = os.path.join(model_filepath, 'fine-tuned-model')
        logging.info("loading the LORA adapter onto the base model from {}".format(fine_tuned_model_filepath))
        model.load_adapter(fine_tuned_model_filepath)
    else:
        fine_tuned_model_filepath = os.path.join(model_filepath, 'fine-tuned-model')
        logging.info("Loading full fine tune checkpoint into cpu from {}".format(fine_tuned_model_filepath))
        model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_filepath, device_map = "auto", trust_remote_code=True, torch_dtype=torch_dtype, local_files_only=True)
        # model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_filepath, trust_remote_code=True, attn_implementation="flash_attention_2", torch_dtype=torch_dtype)

    model.eval()

    tokenizer_filepath = os.path.join(model_filepath, 'tokenizer')
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_filepath)

    return model, tokenizer

In [7]:
import os, json, logging, torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def _two_gpu_max_memory(headroom_gb=2):
    """
    Reserve headroom so HF sharding MUST split across both 16GB T4s.
    """
    if not torch.cuda.is_available():
        return None
    n = torch.cuda.device_count()
    cap = f"{16 - headroom_gb}GiB"  # e.g., "14GiB"
    return {i: cap for i in range(n)}

def _common_from_pretrained_kwargs():
    """
    Settings that reduce both CPU and GPU peak memory and use a lean attention impl.
    """
    kw = dict(
        trust_remote_code=True,
        local_files_only=True,
        torch_dtype=torch.float16,     # T4 → FP16
        low_cpu_mem_usage=True,        # streaming load
        offload_state_dict=True,       # avoid CPU spikes
        attn_implementation="sdpa",    # available by default on Kaggle
    )
    mm = _two_gpu_max_memory(headroom_gb=2)
    if mm and torch.cuda.device_count() > 1:
        kw["device_map"] = "auto"
        kw["max_memory"] = mm
        # Optional if host RAM is tight:
        # kw["offload_folder"] = "/kaggle/working/offload"
    else:
        kw["device_map"] = {"": 0}
    return kw

def load_model_and_tokenizer(model_dir: str, merge_lora: bool = True):
    """
    Robust loader for full fine-tunes or LoRA adapters stored under `model_dir`.
    Expects:
      - reduced-config.json with {"use_lora": <bool>, ...}
      - For LoRA: base-model/, fine-tuned-model/
      - For full FT: fine-tuned-model/
      - tokenizer/ with tokenizer files
    Returns: (model, tokenizer)
    """
    conf_path = os.path.join(model_dir, "reduced-config.json")
    logging.info(f"Loading config: {conf_path}")
    with open(conf_path, "r") as fh:
        cfg = json.load(fh)

    kw = _common_from_pretrained_kwargs()

    if cfg.get("use_lora", False):
        base_dir = os.path.join(model_dir, "base-model")
        lora_dir = os.path.join(model_dir, "fine-tuned-model")

        logging.info(f"Loading base model: {base_dir}")
        model = AutoModelForCausalLM.from_pretrained(base_dir, **kw)
        logging.info(f"Attaching LoRA adapter: {lora_dir}")
        # If PeftModel is missing, use .load_adapter if available
        try:
            model = PeftModel.from_pretrained(model, lora_dir, is_trainable=False)  # type: ignore
        except Exception:
            model.load_adapter(lora_dir)

    else:
        ft_dir = os.path.join(model_dir, "fine-tuned-model")
        logging.info(f"Loading full fine-tuned model: {ft_dir}")
        model = AutoModelForCausalLM.from_pretrained(ft_dir, **kw)

    # Tokenizer hygiene
    tok_dir = os.path.join(model_dir, "tokenizer")
    tokenizer = AutoTokenizer.from_pretrained(tok_dir, use_fast=True, local_files_only=True)
    if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"  # better for causal LMs with dynamic padding

    # Runtime memory knobs for your gradient-based rollout
    model.eval()
    if hasattr(model.config, "use_cache"):
        model.config.use_cache = False  # reduce KV/activation memory during your search

    # Optional: quick sanity check of sharding
    try:
        print(getattr(model, "hf_device_map", "no device map"))
    except Exception:
        pass

    return model, tokenizer

def download_and_load(file_id, output_filename, load_model_path):
    """
    Wrapper that uses your existing helpers:
      - clear_memory(), download_file_from_google_drive(), process_paths()
    """
    clear_memory(verbose=False)

    _ = download_file_from_google_drive(
        file_id=file_id,
        output_dir="/kaggle/tmp",
        output_filename=output_filename,
        quiet=False
    )

    process_paths(paths=["/kaggle/tmp"], recursive=True, max_depth=None)

    model, tokenizer = load_model_and_tokenizer(load_model_path, merge_lora=True)
    return model, tokenizer


In [8]:
model, tokenizer = download_and_load(
    file_id="1lwC9JLRu4Z4SSQwjNtetAymStPqQeaDc",
    output_filename="model0.tar.gz",
    load_model_path="/kaggle/tmp/id-00000000"
)

2025-10-30 13:58:10.687060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761832691.102626     118 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761832691.217886     118 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Downloading...
From (original): https://drive.google.com/uc?id=1lwC9JLRu4Z4SSQwjNtetAymStPqQeaDc
From (redirected): https://drive.google.com/uc?id=1lwC9JLRu4Z4SSQwjNtetAymStPqQeaDc&confirm=t&uuid=e148cdaf-63a9-44db-a4e0-f142df36d9e5
To: /kaggle/tmp/model0.tar.gz
100%|██████████| 10.6G/10.6G [01:34<00:00, 112MB/s] 


Download successful! File saved to: /kaggle/tmp/model0.tar.gz
File size: 10092.92 MB
Processing directory: /kaggle/tmp
Extracting: /kaggle/tmp/model0.tar.gz
Deleted compressed file: /kaggle/tmp/model0.tar.gz
Total .tar.gz files processed: 1


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

{'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 0, 'model.layers.11': 0, 'model.layers.12': 0, 'model.layers.13': 0, 'model.layers.14': 0, 'model.layers.15': 0, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 1, 'model.layers.26': 1, 'model.layers.27': 1, 'model.layers.28': 1, 'model.layers.29': 1, 'model.layers.30': 1, 'model.layers.31': 1, 'model.norm': 1, 'model.rotary_emb': 1, 'lm_head': 1}


In [9]:
from torch.nn import functional as F

def entropy_loss(batch_logits):
    log_probs = F.log_softmax(batch_logits, dim=-1)
    probs = log_probs.exp()
    entropy = -(probs * log_probs).sum(dim=-1)
    return entropy.mean()

In [10]:
from datasets import load_dataset
from torch.utils.data import DataLoader
import os, torch

def load_prompts(tokenizer, args):
    # Load once; keep raw text to avoid huge pre-tokenized tensors in RAM
    ds = load_dataset("tatsu-lab/alpaca", split="train", cache_dir=args["data_dir"])

    def collate(batch):
        texts = [ex["instruction"] for ex in batch]
        enc = tokenizer(
            texts,
            padding=True,                  # dynamic padding to batch max
            truncation=True,
            max_length=args["max_length"],
            return_tensors="pt"
        )
        return enc["input_ids"], enc["attention_mask"]

    num_workers = max(2, os.cpu_count() // 2)
    return DataLoader(
        ds,
        batch_size=args["batch_size"],
        shuffle=False,
        pin_memory=True,
        num_workers=num_workers,
        persistent_workers=True,
        collate_fn=collate
    )

In [11]:
args = {
    "data_dir": "kaggle/working/data",
    "max_length": 512,
    "batch_size": 16
}
dataloader = load_prompts(tokenizer, args)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [12]:
bs   = getattr(dataloader, "batch_size", 1)
nw   = getattr(dataloader, "num_workers", 0)
pin  = getattr(dataloader, "pin_memory", "cpu" != "cpu")
pers = getattr(dataloader, "persistent_workers", False)
print(f"DataLoader settings - batch_size: {bs}, num_workers: {nw}, pin_memory: {pin}, persistent_workers: {pers}")

DataLoader settings - batch_size: 16, num_workers: 2, pin_memory: True, persistent_workers: True


In [33]:
from torch import nn

def find_embedding_layer(model):
    """
    Attempts to find the embedding layer in an arbitrary PyTorch model.
    
    Parameters:
    - model: A PyTorch model (nn.Module) or Hugging Face model.
    
    Returns:
    - embedding_layer: The nn.Embedding layer (or None if not found).
    - path: The attribute path to the embedding layer (e.g., 'embeddings.word_embeddings').
    """
    # Check if the model has a get_input_embeddings method (common in Hugging Face)
    if hasattr(model, 'get_input_embeddings'):
        emb = model.get_input_embeddings()
        if isinstance(emb, nn.Embedding):
            return emb, 'get_input_embeddings()'
    
    # Iterate through all named modules to find an embedding layer
    for name, module in model.named_modules():
        if isinstance(module, nn.Embedding):
            return module, name
    
    # If no embedding layer is found, return None
    return None, None

def freeze_except_embeddings(model, emb_layers):
    """
    Freezes all model parameters except the weights of specified embedding layers.
    
    Parameters:
    - model: PyTorch model (nn.Module).
    - emb_layers: Single nn.Embedding layer or list of nn.Embedding layers to keep unfrozen.
    
    Returns:
    - None
    """
    # Convert single embedding layer to list for generality
    if isinstance(emb_layers, nn.Embedding):
        emb_layers = [emb_layers]
    
    # Validate that emb_layers are part of the model
    model_params = set(model.parameters())
    for emb_layer in emb_layers:
        if not isinstance(emb_layer, nn.Embedding):
            raise ValueError(f"Expected nn.Embedding, got {type(emb_layer)}")
        if emb_layer.weight not in model_params:
            raise ValueError("Embedding layer weight is not part of the model's parameters")
    
    # Get set of embedding weights to keep unfrozen
    emb_weights = set(emb_layer.weight for emb_layer in emb_layers)
    
    # Freeze parameters and clear gradients
    for name, param in model.named_parameters():
        if param in emb_weights:
            param.requires_grad = True  # Ensure embedding weights are trainable
        else:
            param.requires_grad = False
            param.grad = None  # Clear gradients to save memory
    
    # Verify embedding layers remain unfrozen
    for emb_layer in emb_layers:
        assert emb_layer.weight.requires_grad, f"Embedding layer {emb_layer} was unexpectedly frozen"

In [40]:
from torch.cuda.amp import autocast

def _one_step(model, embeddings, attention_mask, amp_dtype=torch.float16):
    with autocast(dtype=amp_dtype):
        out = model(inputs_embeds=embeddings, attention_mask=attention_mask)
        logits = out.logits[:, -1, :]
    return logits

def _build_prompt_vocab_mask(input_ids, vocab_size, special_ignores=None, device=None):
    """
    Build a (B, V) binary mask where mask[b, v] = 1 iff token v appears in prompt b,
    excluding special tokens in `special_ignores`.
    """
    B = input_ids.size(0)
    mask = torch.zeros(B, vocab_size, dtype=torch.float32, device=device)
    for b in range(B):
        ids = input_ids[b]
        if special_ignores:
            for tok in special_ignores:
                if tok is not None:
                    ids = ids[ids != tok]
        uniq = ids.unique()
        mask[b, uniq] = 1.0
    return mask


def compute_loss(
    model,
    emb_layer,
    embeddings,          # (B, L, E), requires grad
    attention_mask,      # (B, L)
    n_tokens,
    amp_dtype=torch.float16,
    input_ids_prompt=None,           # (B, L) original prompt token ids
    tokenizer=None,                  # to get pad/eos/bos if needed
    copy_weight=0.3,                 # penalize copying prompt tokens
    self_repeat_weight=0.5           # penalize repeating the last generated token
):
    """
    Autoregressive roll-out for n_tokens. Adds differentiable penalties that
    discourage copying input and repeating the last token.
    """
    dev = embeddings.device
    B = embeddings.size(0)
    V = emb_layer.weight.size(0)

    # ---- Build prompt-vocab mask (B, V) once ----
    if input_ids_prompt is None:
        # We can approximate from embeddings' first L positions, but better to pass input_ids_prompt
        raise ValueError("Pass input_ids_prompt to enable anti-copy loss.")
    special_ignores = []
    if tokenizer is not None:
        special_ignores = [tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.bos_token_id]
    prompt_vocab_mask = _build_prompt_vocab_mask(
        input_ids_prompt.to(dev, non_blocking=True),
        vocab_size=V,
        special_ignores=special_ignores,
        device=dev
    )  # (B, V), float32

    total_main = 0.0
    total_copy = 0.0
    total_repeat = 0.0

    last_gen_token = None  # track argmax token from previous step (per batch)


    for t in range(n_tokens):
        logits = _one_step(model, embeddings, attention_mask, amp_dtype)  # (B, V)
        probs = torch.softmax(logits, dim=-1)           # (B, V)

        # ---- Main task loss (entropy of the distribution at this step) ----
        step_main = entropy_loss(logits)                     # scalar

        # ---- Anti-copy: probability mass on any prompt token ----
        # (B,V)·(B,V) -> (B,), then mean
        copy_mass = (probs * prompt_vocab_mask).sum(dim=-1).mean()  # scalar
        step_copy = copy_weight * copy_mass

        # ---- Anti-self-repeat: probability mass on last generated token ----
        if last_gen_token is not None and self_repeat_weight > 0.0:
            # gather probs for last_gen_token per batch
            p_repeat = probs.gather(1, last_gen_token.view(-1, 1)).mean()
            step_repeat = self_repeat_weight * p_repeat
        else:
            step_repeat = 0.0

        # Accumulate losses
        # earlier steps are treated as constants via .detach()
        if t < n_tokens - 1:
            total_main   = total_main   + step_main.detach()
            total_copy   = total_copy   + step_copy.detach()
            total_repeat = total_repeat + (step_repeat if isinstance(step_repeat, float) else step_repeat.detach())
        else:
            total_main   = total_main   + step_main
            total_copy   = total_copy   + step_copy
            total_repeat = total_repeat + (step_repeat if isinstance(step_repeat, float) else step_repeat)

        # ---- Roll the context forward (no sampling; use expected embedding) ----
        # Choose next token id (argmax) only for bookkeeping (no grad needed)
        with torch.no_grad():
            next_ids = torch.argmax(probs, dim=-1)         # (B,)
            last_gen_token = next_ids                      # store for next step
            # expected embedding for differentiable roll-out
            exp_embed = probs @ emb_layer.weight.to(dev)   # (B, E)
        embeddings = torch.cat([embeddings, exp_embed.unsqueeze(1)], dim=1)  # (B, L+t+1, E)
        attention_mask = torch.cat(
            [attention_mask, torch.ones((B,1), dtype=attention_mask.dtype, device=dev)],
            dim=1
        )

    total_loss = total_main + total_copy + total_repeat
    return total_loss


In [35]:
import torch
from tqdm import tqdm


def _first_device_of_embedding(model):
    emb, _ = find_embedding_layer(model)
    if emb is None:
        raise RuntimeError("Could not find an nn.Embedding in the model.")
    return emb, emb.weight.device

def find_best_flip(
    model, batch, max_length_tokens, batch_id, topk, vocab_chunk
):
    """
    Memory-lean search:
      - compute grad wrt embeddings only for the final step,
      - process each sample independently,
      - chunk vocab to avoid allocating (V×L) at once.
    """
    model.eval()
    emb_layer, first_dev = _first_device_of_embedding(model)

    input_ids = batch[0].to(first_dev, non_blocking=True)      # (B, L)
    attention_mask = batch[1].to(first_dev, non_blocking=True) # (B, L)

    # Build initial embeddings (needs to be tracked)
    freeze_except_embeddings(model, emb_layer)
    embeddings = emb_layer(input_ids)                          # (B, L, E)
    embeddings.retain_grad()


    # Compute loss (only last step tracked)
    loss = compute_loss(
        model, emb_layer, embeddings, attention_mask,
        n_tokens=max_length_tokens,
        amp_dtype=torch.float16,
        input_ids_prompt=input_ids,          # <--- add this
        tokenizer=tokenizer,                 # (optional; helps ignore pad/eos)
        copy_weight=0.3,                     # tune
        self_repeat_weight=0.5               # tune
    )



    # Backprop to get grads wrt embeddings
    model.zero_grad(set_to_none=True)
    loss.backward()

    grads = embeddings.grad.detach()   # (B, L, E)
    embeds_det = embeddings.detach()   # (B, L, E)
    del embeddings, loss
    torch.cuda.empty_cache()

    # We'll need vocab embeddings on GPU, but in CHUNKS
    results = []

    B, L, E = grads.shape
    V = emb_layer.weight.size(0)
    dev = first_dev

    # s_i = (g_i · v_i) per position, per sample -> (B, L)
    s_i = (grads * embeds_det).sum(dim=2)  # (B, L)

    # If you have attention_mask available in this scope, mask out padding positions now.
    # Make sure it's on the same device as scores later.
    # Uncomment if attention_mask exists:
    attn_mask = attention_mask.to(dev, non_blocking=True)  # (B, L)
    mask_b1l = (attn_mask == 0).unsqueeze(1)               # (B, 1, L)

    # Running accumulators across vocab chunks
    if topk == 1:
        best_vals = torch.full((B,), float("inf"), device=dev, dtype=grads.dtype)
        best_flat_idx = torch.full((B,), -1, device=dev, dtype=torch.long)
    else:
        vals_keep = None  # (B, k_accum)
        idx_keep  = None  # (B, k_accum)

    offset = 0  # how many (vocab positions * L) we've traversed so far

    for start in range(0, V, vocab_chunk):
        end = min(start + vocab_chunk, V)
        vocab_slice = emb_layer.weight[start:end].to(dev, non_blocking=True)  # (vchunk, E)
        # scores = (B, vchunk, L) = (vchunk, E) @ (B, E, L)
        # using einsum handles broadcasting cleanly
        scores = torch.einsum("ve,ble->bvl", vocab_slice, grads)  # (B, vchunk, L)

        # subtract s_i across L
        scores = scores - s_i.unsqueeze(1)  # (B, 1, L) broadcast

        # Optional: mask padding positions so they are never picked
        # if 'mask_b1l' defined above:
        scores = scores.masked_fill(mask_b1l, float("inf"))

        # Flatten chunk per sample to shape (B, vchunk*L)
        flat = scores.reshape(B, -1)

        if topk == 1:
            # best of this chunk per sample
            chunk_vals, chunk_idx = torch.min(flat, dim=1)  # (B,)
            # where better than current best, update
            update = chunk_vals < best_vals
            best_vals = torch.where(update, chunk_vals, best_vals)
            best_flat_idx = torch.where(update, chunk_idx + offset, best_flat_idx)
        else:
            k_here = min(topk, flat.size(1))
            chunk_vals, chunk_idx = torch.topk(flat, k=k_here, largest=False, dim=1)  # (B, k_here)
            chunk_idx = chunk_idx + offset  # globalize indices

            if vals_keep is None:
                vals_keep, idx_keep = chunk_vals, chunk_idx
            else:
                # Merge with accumulated bests and reselect top-k
                vals_keep = torch.cat([vals_keep, chunk_vals], dim=1)  # (B, k_accum + k_here)
                idx_keep  = torch.cat([idx_keep,  chunk_idx],  dim=1)  # (B, k_accum + k_here)
                # Re-topk across the concatenated candidates
                k_sel = min(topk, vals_keep.size(1))
                sel_vals, sel_pos = torch.topk(vals_keep, k=k_sel, largest=False, dim=1)  # (B, k_sel)
                # Gather the matching global indices
                batch_ids = torch.arange(B, device=dev).unsqueeze(1).expand_as(sel_pos)
                idx_keep = idx_keep[batch_ids, sel_pos]  # (B, k_sel)
                vals_keep = sel_vals

        # free chunk temporaries
        del vocab_slice, scores, flat
        torch.cuda.empty_cache()

        # advance flat offset by the chunk length (vchunk * L)
        offset += (end - start) * L

    # ----- Convert batched results to your result list -----
    if topk == 1:
        # best_flat_idx encodes (v, pos) as: idx = v * L + pos
        best_v   = (best_flat_idx // L).tolist()
        best_pos = (best_flat_idx %  L).tolist()
        best_val = best_vals.tolist()

        for b in range(B):
            results.append({
                "best_position": int(best_pos[b]),
                "best_vocab_index": int(best_v[b]),
                "min_score": float(best_val[b]),
                "sample_id": int(b),
                "batch_id": int(batch_id)
            })
    else:
        # idx_keep/vals_keep are (B, topk)
        v_idx = (idx_keep // L).tolist()
        pos_i = (idx_keep %  L).tolist()
        vals  =  vals_keep.tolist()

        for b in range(B):
            pairs = [{"best_position": int(pos_i[b][j]),
                    "best_vocab_index": int(v_idx[b][j]),
                    "min_score": float(vals[b][j]),
                    "sample_id": int(b),
                    "batch_id": int(batch_id)}
                    for j in range(len(v_idx[b]))]
            results.extend(pairs)


    del grads, embeds_det, input_ids, attention_mask
    torch.cuda.empty_cache(); gc.collect()
    return results


In [36]:
check_memory()

Free GPU Memory: 8.44 GB
Total GPU Memory: 14.74 GB


In [37]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn.functional as F
from collections import defaultdict


def get_top_k_by_min_score(dict_list, k):
    return sorted(dict_list, key=lambda x: float(x["min_score"]))[:k]

def make_varlen_collate(tokenizer):
    pad_id = tokenizer.pad_token_id
    if pad_id is None:
        # common for LLaMA-style tokenizers
        tokenizer.pad_token_id = tokenizer.eos_token_id
        pad_id = tokenizer.pad_token_id
    def _collate_varlen(samples):
        """
        samples: list of tuples (ids_1D, mask_1D) with possibly varying lengths.
        Pads within the batch using each sample's last token id as pad value; mask pads with 0.
        """
        if not samples:
            return torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long)
        max_len = max(s[0].numel() for s in samples)
        ids_out, msk_out = [], []
        for ids, msk in samples:
            # pad_val = int(ids[-1].item()) if ids.numel() > 0 else 0
            if ids.numel() < max_len:
                ids = F.pad(ids, (0, max_len - ids.numel()), value=pad_id)
                msk = F.pad(msk, (0, max_len - msk.numel()), value=0)
            ids_out.append(ids)
            msk_out.append(msk)
        return torch.stack(ids_out, dim=0), torch.stack(msk_out, dim=0)
    return _collate_varlen
    

def apply_topk_results_to_inputs(
    original_dataloader,
    top_results,
    batch_cache,                 # REQUIRED: dict {batch_id: (ids_cpu, mask_cpu)}
    device="cpu",
):
    # Mirror original loader knobs
    bs   = getattr(original_dataloader, "batch_size", 1)
    nw   = getattr(original_dataloader, "num_workers", 0)
    pin  = getattr(original_dataloader, "pin_memory", device != "cpu")
    pers = getattr(original_dataloader, "persistent_workers", False)

    # Group flips by batch for efficient lookup
    flips_by_batch = defaultdict(list)  # batch_id -> list[(sample_id, pos, tok)]
    for r in top_results:
        flips_by_batch[int(r["batch_id"])].append(
            (int(r["sample_id"]), int(r["best_position"]), int(r["best_vocab_index"]))
        )
    # print(f"flips: {flips_by_batch.items()}")
    # print(f"cache: {batch_cache}")

    edited_samples = []
    for b_id, flips in flips_by_batch.items():
        ids_cpu, msk_cpu = batch_cache[b_id]  # (B,L), (B,L)
        # print(f"lol, {flips}")
        for s, p, t in flips:
            ids_1d = ids_cpu[s].clone()
            msk_1d = msk_cpu[s].clone()
            L_s = ids_1d.numel()
            if 0 <= p < L_s:
                ids_1d[p] = t
                msk_1d[p] = 1
                edited_samples.append((ids_1d, msk_1d))
    print(f"edited_samples: {len(edited_samples)}")

    # Return loader over ONLY edited samples; safe even if empty
    collate_fn = make_varlen_collate(tokenizer)
    return DataLoader(
        edited_samples,
        batch_size=bs,
        shuffle=False,
        pin_memory=pin,
        num_workers=nw,
        persistent_workers=pers if nw > 0 else False,
        collate_fn=collate_fn
    ), edited_samples

In [38]:
import os, torch, json

def save_round_data(edited_samples, top_results, round_id, out_dir="/kaggle/working/rounds"):
    os.makedirs(out_dir, exist_ok=True)

    # Save tensors
    tensor_path = os.path.join(out_dir, f"round_{round_id:03d}_samples.pt")
    torch.save(edited_samples, tensor_path)

    # Extract indices info (sample_id, batch_id, position, vocab_index)
    meta = [
        {
            "sample_id": int(r["sample_id"]),
            "batch_id": int(r["batch_id"]),
            "position": int(r["best_position"]),
            "vocab_index": int(r["best_vocab_index"]),
            "score": float(r["min_score"])
        }
        for r in top_results
    ]

    # Save metadata
    meta_path = os.path.join(out_dir, f"round_{round_id:03d}_meta.json")
    with open(meta_path, "w") as f:
        json.dump(meta, f, indent=2)

    print(f"Saved round {round_id} data: {len(edited_samples)} samples → {tensor_path}")

In [ ]:
def load_from_saved_samples(tokenizer, dataloader, round_filepath):
    print(f"Loading dataloader from previous round file: {round_filepath}")
    edited_samples = torch.load(round_filepath)
    # Mirror original loader knobs
    bs   = getattr(dataloader, "batch_size", 1)
    nw   = getattr(dataloader, "num_workers", 0)
    pin  = getattr(dataloader, "pin_memory", False)
    pers = getattr(dataloader, "persistent_workers", False)
    collate_fn = make_varlen_collate(tokenizer)
    dataloader = DataLoader(
        edited_samples,
        batch_size=bs,
        shuffle=False,
        pin_memory=pin,
        num_workers=nw,
        persistent_workers=pers if nw > 0 else False,
        collate_fn=collate_fn
    )

In [ ]:
import torch, gc

rounds = 10
topk_values = [26858, 14427, 7750, 4163, 2236, 1201, 645, 347, 186, 100]

for r in range(rounds):
    batch_cache = {}
    batch_id = 0
    results = []
    # If /kaggle/working/hotflip/round_XXX_samples.pt exists, load that instead of the original dataloader
    round_filepath = f"/kaggle/working/hotflip/rounds/round_{r:03d}_samples.pt"
    next_round_filepath = f"/kaggle/working/hotflip/rounds/round_{r+1:03d}_samples.pt"
    if os.path.exists(next_round_filepath):
        print(f"Next round file already exists: {next_round_filepath}, skipping round {r}")
        continue
    if os.path.exists(round_filepath):
        dataloader = load_from_saved_samples(tokenizer, dataloader, round_filepath)
        continue
    for batch in tqdm(dataloader, desc=f"=== Round {r+1}/{rounds} ==="):
        _, first_dev = _first_device_of_embedding(model)
        batch = (batch[0].to(first_dev, non_blocking=True),
                batch[1].to(first_dev, non_blocking=True))
        batch_cache[batch_id] = (batch[0].cpu(), batch[1].cpu())

        results.extend(find_best_flip(
            model=model,
            batch=batch,
            max_length_tokens=10,
            batch_id=batch_id,
            topk=5,
            vocab_chunk=8192,         # tune: 4096–16384 depending on VRAM
        ))
        batch_id += 1

        # free per-iteration junk
        del batch
        torch.cuda.empty_cache()
        print(f"batch {batch_id} done")

    # Keep only the top-k and rebuild the loader
    print(f"results before: {len(results)}")
    results = get_top_k_by_min_score(results, k=topk_values[int(r)])
    print(f"results after: {len(results)}")
    dataloader, edited_samples = apply_topk_results_to_inputs(dataloader, results, batch_cache)

    # TODO save edited_samples in a proper file, indicating the round as well
    save_round_data(edited_samples, results, int(r))

    # housekeeping
    torch.cuda.empty_cache(); gc.collect()

print(f"final results: {results}")

=== Round 1/10 ===:   0%|          | 1/3251 [00:05<4:42:43,  5.22s/it]

batch 1 done


=== Round 1/10 ===:   0%|          | 2/3251 [00:09<4:04:05,  4.51s/it]

batch 2 done


=== Round 1/10 ===:   0%|          | 3/3251 [00:14<4:13:04,  4.67s/it]

batch 3 done


=== Round 1/10 ===:   0%|          | 4/3251 [00:18<4:03:50,  4.51s/it]

batch 4 done


=== Round 1/10 ===:   0%|          | 4/3251 [00:18<4:14:11,  4.70s/it]


KeyboardInterrupt: 

In [ ]:
check_memory()

Free GPU Memory: 1.95 GB
Total GPU Memory: 14.74 GB
